In [38]:
import psycopg2
import pandas as pd
from utils import *

In [39]:
conn = psycopg2.connect(
    host = "codd04.research.northwestern.edu",
    database = "postgres",
    user = "cpdbstudent",
    password = "DataSci4AI",
    port = 5433
)
cursor = conn.cursor()

In [40]:
cursor.execute("""
    SELECT * FROM trr_trrstatus_refresh;
""")
results = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]

In [41]:
df = pd.DataFrame(results, columns=colnames)
df

,officer_rank,officer_star,status,status_datetime,officer_age,officer_first_name,officer_middle_initial,officer_last_name,officer_gender,officer_race,officer_birth_year,officer_appointed_date,officer_unit_at_incident,trr_report_id
0,OFFICER,4502,None,2016-04-12 11:53:00-05,36.0,SYLWIA,None,REJMAN,F,WHITE,None,11-30-12,None,96449
1,SERGEANT,1543,None,2016-04-13 12:02:00-05,34.0,NIKI,None,CAMPUS,F,WHITE,None,08-29-05,None,96449
2,LIEUTENANT,330,None,2016-04-13 02:36:00-05,55.0,STEPHANIE,None,STUART,F,BLACK,None,11-07-88,None,96449
3,OFFICER,17679,None,2016-04-12 11:49:00-05,31.0,EUGENE,None,VANN,M,WHITE HISPANIC,None,11-24-14,None,96450
4,SERGEANT,1543,None,2016-04-12 11:50:00-05,34.0,NIKI,None,CAMPUS,F,WHITE,None,08-29-05,None,96450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63450,Sergeant of Police,None,SUBMITTED,2020-06-06 12:39:00-05,NaN,CHARLIE,S,JOHNSON,M,BLACK,1980,2002-OCT-28,2,6314297
63451,Lieutenant of Police,None,APPROVED,2020-06-07 01:17:00-05,NaN,TAMARA,B,MARGOLIS,F,WHITE,1974,1998-JUL-27,2,6314297
63452,Police Officer,None,SUBMITTED,2020-06-06 02:20:00-05,NaN,KARLA,L,ESPINOZA,F,WHITE HISPANIC,1980,2016-AUG-29,25,6352461
63453,Sergeant of Police,None,SUBMITTED,2020-06-06 05:28:00-05,NaN,MARK,E,FLECHSIG,M,WHITE,1969,1998-MAY-04,25,6352461


In [42]:
df['officer_birth_year'] = to_int(df['officer_birth_year'])

In [43]:
# convert to datetime
df['status_datetime'] = pd.to_datetime(df['status_datetime'])

In [44]:
# remove redacted
df['officer_appointed_date'].replace('REDACTED', None, inplace=True)
# convert to dates
df['officer_appointed_date'] = pd.to_datetime(df['officer_appointed_date'])

In [45]:
df['officer_first_name'] = df['officer_first_name'].str.title()
df['officer_last_name'] = df['officer_last_name'].str.title()
# TODO remove suffix

In [47]:
# reconcile officer races
reconcile_race(df, 'officer_race')
pd.unique(df['officer_race'])

array(['WHITE', 'BLACK', 'HISPANIC', 'ASIAN/PACIFIC ISLANDER',
       'NATIVE AMERICAN/ALASKAN NATIVE'], dtype=object)

In [48]:
pd.unique(df['officer_gender'])

array(['F', 'M'], dtype=object)